In [3]:
from os import sys, path
sys.path.append(path.dirname(path.dirname(path.abspath('__file__'))))

import pprint

import pandas as pd
import numpy as np

import h5py
import progressbar
from pack.utils import delete_directory_contents
# import sys
# import warnings
#
# # if not sys.warnoptions:
# warnings.simplefilter("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)

/home/kasper/PycharmProjects/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
def get_match_indexes(data_h5, teams):
    """
    This function returns the unique index for the pairs.
    """

    arrays = []
    for team in teams[:]:
        ix = data_h5[team][:].T[:,0]
        arrays.append(ix)
    uni = np.unique(np.concatenate(arrays)).astype(int)
    return uni


def get_elo_home_and_away(data_h5, row, ix):
    """
    This functions takes the data from the pair. iterates over the main data
    and finds the coresponding elo ratings based on the side the team is on.
    """
    hdhome = data_h5[row.home_team][:].T
    hdaway = data_h5[row.away_team][:].T
    elohome = hdhome[np.where(hdhome==ix)[1],1]
    eloaway = hdaway[np.where(hdaway==ix)[1],1]
    return elohome, eloaway

In [5]:
print('[LOADING...]: Main DataFrame.')
df = pd.read_csv('/home/kasper/Dropbox/Scrapping/soccerway/csv/final_data_soccerway.csv', index_col='Unnamed: 0')

pred_data = pd.read_csv('../data/predict.csv', index_col='Unnamed: 0')
df_teams = pred_data[['home_team','away_team']].values

[LOADING...]: Main DataFrame.


/home/kasper/PycharmProjects/venv/lib/python3.5/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
for p, pair in enumerate(df_teams[:]):

    print('[INFO]: pairs: ', pair)

    h5f = h5py.File('./elo_temp/elo_pairs_' + str(p) + '.h5', 'r')
    # TODO: rename champ to pair since that what it is
    champ = list(h5f.keys())[0]
    # NOTE: do i need to convert this to a list since i iterate over it?
    teams = list(h5f[champ].keys())
    data = h5f[champ]
    print('[INFO]: getting indexes')
    uni = get_match_indexes(data_h5=data, teams=teams)

    da = df.loc[uni][['home_team','away_team']]
    da['res_index'] = uni
    

    print('[INFO]: finding ratings.')
    elos = np.zeros((da.shape[0], 2))
    for i, (ix, row) in enumerate(da.iloc[:].iterrows()):
        try:
            elohome, eloaway = get_elo_home_and_away(row=row, ix=ix, data_h5=data)
            elos[i,0] = elohome
            elos[i,1] = eloaway
        except Exception as e:
            elos[i,0] = np.nan
            elos[i,1] = np.nan

    elos_df = pd.DataFrame(elos, columns=['EH','EA'], index=da.index)

    final = pd.concat([da,elos_df],axis=1)

    final.to_csv('./elo_goals_data_for_classifier/data_'+str(p)+'.csv')
    h5f.close()

In [ ]:
shared_dict = {
    # big dict with ~10k keys, each with a list of dicts
}

balancer = engines.load_balanced_view()

with engines[:].sync_imports(): # your 'view' variable 
    import pandas as pd
    import ujson as json

engines[:].push(shared_dict)

results = balancer.map(lambda i: (i, my_func(i)), id)
results_data = results.get()

In [1]:
import ipyparallel as ipp

cluster = ipp.Client(profile='ssh')
# cluster = ipp.Client()

view = cluster.load_balanced_view()


In [ ]:
for i in range(10):
    for j in range(10):
        # This line submits the tasks for parallel computation.
        ar = view.apply_async(test, i,j)
        async_results.append(ar)

cluster.wait_interactive(async_results) 